# pynhm Errors against PRMS

Using the `test_data` multi-hru domains (drb_2yr or ucb_2yr), we will setup and run a pynhm model (with PRMS inputs). We will then compare results against PRMS outputs. 

In [ ]:
import pathlib as pl
import numpy as np
import param
import pynhm
import xarray as xr

import panel_errors_helpers

## Model setup and run

In [ ]:
domain = 'ucb_2yr'
processes = [
    pynhm.PRMSSolarGeometry, 
    # pynhm.PRMSAtmosphere, 
    # pynhm.PRMSCanopy,
    # pynhm.PRMSSnow,
    # pynhm.PRMSRunoff,
    # pynhm.PRMSSoilzone,
    # pynhm.PRMSGroundwater,
    # pynhm.PRMSChannel,
]

In [ ]:
repo_dir = pynhm.constants.__pynhm_root__.parent
domain_dir = repo_dir / f'test_data/{domain}/'
gis_dir = repo_dir / f'examples/pynhm_gis/{domain}'

run_dir = repo_dir / f"pynhm/analysis/runs/{domain}_{'_'.join([proc.__name__ for proc in processes])}"  #change this
input_dir_all = run_dir / 'inputs'

parameter_file = domain_dir / 'myparam.param'
control_file = domain_dir / 'control.test'
parameters = pynhm.PRMSParameters.load(parameter_file)
control = pynhm.Control.load(control_file, params=parameters)

# to re-run, delete the run_dir
run_flag = False
if not run_dir.exists(): 
    run_flag = True
    # Combine all PRMS inputs and outputs in one directory for pynhm input   
    input_dir_all.mkdir(exist_ok=True, parents=True)
    for nc_file in domain_dir.glob('**/*.nc'):
        target = input_dir_all / (nc_file.name)
        if not target.exists():
            target.symlink_to(nc_file)

# instantiate the model
model = pynhm.Model(*processes, control=control, input_dir=input_dir_all, budget_type='warn', netcdf_output_dir=run_dir)

if run_flag:
    model.run(finalize=True)

## Collect and load pynhm output and PRMS equivalents

In [ ]:
import importlib
importlib.reload(panel_errors_helpers)
err_data_hru = np.zeros((model.control.params.parameters['nhru'],)) * np.nan
err_data_seg = np.zeros((model.control.params.parameters['nsegment'],)) * np.nan
panel_errors_helpers.err_panel(model, gis_dir=gis_dir, err_data_hru=err_data_hru, err_data_seg=err_data_seg)

In [ ]:
import sys
sys.path.append('../common')
import process_plot
pp = process_plot.ProcessPlot(gis_dir)

pp.plot_hru('data', model, data=err_data_hru, data_units='whatevers')  # the error data on hru
#pp.plot_seg('data', model, data=err_data_seg, data_units='whatevers') # the error data on segment

In [ ]:
# pp.plot_hru('pkwater_equiv', model) # variable